In [ ]:
!pip install -q datasets
!pip install -q anthropic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 4.8 MB/s eta 0:00:00


In [ ]:
import time
import anthropic
from typing import List, Dict
import os

In [ ]:
os.environ["ANTHROPIC_API_KEY"] = "YOUR_ANTHROPIC_API_KEY"

In [ ]:
def _claudeqa(prompt: str, system_message: str):
    client = anthropic.Anthropic()
    message = client.messages.create(
        model="claude-3-7-sonnet-20250219",
        max_tokens=8192,
        system=system_message,
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ]
    )
    return message.content[0].text, message.usage

In [ ]:
def apiqa(prompt: str, model_name: str, system_message: str, json_format: bool = True):
    completion = None
    while completion is None:
        try:
            if model_name == 'claude-3-7-sonnet-20250219':
                assert json_format == False, "Claude does not support json format"
                completion, usage = _claudeqa(prompt, system_message)
        except Exception as e:
            print(f"Exception: {str(e)}")
            time.sleep(60)
    return completion, usage

In [ ]:
grading_prompt = """You are an AI assistant for grading a science problem.
The user will provide you with the question itself, an attempt made by a student and the correct answer to the problem.
Your job is to judge whether the attempt is correct by comparing it with the correct answer.
If the expected solution concludes with a number or choice, there should be no ambiguity.
If the expected solution involves going through the entire reasoning process, you should judge the attempt based on whether the reasoning process is correct with correct answer if helpful.

The user will provide the attempt and the correct answer in the following format:

# Problem
{problem}

## Attempt
{attempt}

## Correct answer
{solution}

Explain your reasoning, and end your response on a new line with only "Yes" or "No" (without quotes)."""

In [ ]:
from datasets import load_dataset
from hashlib import sha256
import json
import io

ds = load_dataset("simplescaling/s1K-1.1")['train']

def _make_w_io_base(f, mode: str):
    if not isinstance(f, io.IOBase):
        f_dirname = os.path.dirname(f)
        if f_dirname != "":
            os.makedirs(f_dirname, exist_ok=True)
        f = open(f, mode=mode, encoding="utf-8")
    return f


def jdump(obj, f: str, mode="w", indent=4, default=str):
    """Dump a str or dictionary to a file in json format.

    Args:
        obj: An object to be written.
        f: A string path to the location on disk.
        mode: Mode for opening the file.
        indent: Indent for storing json dictionaries.
        default: A function to handle non-serializable entries; defaults to `str`.
    """
    f = _make_w_io_base(f, mode)
    if isinstance(obj, (dict, list)):
        json.dump(obj, f, indent=indent, default=default)
    elif isinstance(obj, str):
        f.write(obj)
    else:
        raise ValueError(f"Unexpected type: {type(obj)}")
    f.close()

def question_hash(question: str) -> str:
    return sha256(question.encode()).hexdigest()[:16]

def add_grading_gemini(example):
    user_prompt = f"## Problem\n{example['question']}\n\n## Attempt\n{example['gemini_attempt']}\n\n## Correct answer\n{example['solution']}"
    # do grading
    grade = None
    while grade is None:
        completion, _ = apiqa(user_prompt, "claude-3-7-sonnet-20250219", grading_prompt, json_format=False)
        if completion.split('\n')[-1] in ["Yes", "No"]:
            grade = completion.split('\n')[-1]
        else:
            print(f"Invalid response: {completion}")
            time.sleep(60)

    example['gemini_grade'] = grade
    example['gemini_grade_reason'] = completion

    jdump(dict(example), f"{question_hash(example['question'])}.json")

    return example


ds = ds.map(add_grading_gemini, num_proc=4)


Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
!mkdir gemini_json
!mv *json gemini_json/

mv: cannot move 'gemini_json' to a subdirectory of itself, 'gemini_json/gemini_json'


In [ ]:
from collections import Counter
Counter(ds['gemini_grade'])

Counter({'Yes': 536, 'No': 464})

In [ ]:
def add_grading_deepseek(example):
    user_prompt = f"## Problem\n{example['question']}\n\n## Attempt\n{example['deepseek_attempt']}\n\n## Correct answer\n{example['solution']}"
    # do grading
    grade = None
    while grade is None:
        completion, _ = apiqa(user_prompt, "claude-3-7-sonnet-20250219", grading_prompt, json_format=False)
        if completion.split('\n')[-1] in ["Yes", "No"]:
            grade = completion.split('\n')[-1]
        else:
            print(f"Invalid response: {completion}")
            time.sleep(60)

    example['deepseek_grade'] = grade
    example['deepseek_grade_reason'] = completion

    jdump(dict(example), f"{question_hash(example['question'])}.json")

    return example


ds = ds.map(add_grading_deepseek, num_proc=8)

Map (num_proc=8):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
ds.push_to_hub("simplescaling/s1K-1.1")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/simplescaling/s1K-1.1/commit/96c411f1fe4c49d20f0e2a1565f61e1a28b0b84d', commit_message='Upload dataset', commit_description='', oid='96c411f1fe4c49d20f0e2a1565f61e1a28b0b84d', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/simplescaling/s1K-1.1', endpoint='https://huggingface.co', repo_type='dataset', repo_id='simplescaling/s1K-1.1'), pr_revision=None, pr_num=None)

In [ ]:
from collections import Counter
Counter(ds['deepseek_grade'])

Counter({'Yes': 630, 'No': 370})